## Compute lattice constant using the current potential.

In [ ]:
from ase.build import bulk
from ase.calculators.emt import EMT
from ase.db import connect
from ase.eos import calculate_eos


atoms = bulk('Cu', 'fcc')
atoms.calc = EMT()
eos = calculate_eos(atoms)
v, e, B = eos.fit()  # find minimum
# Do one more calculation at the minimum and write to database:
atoms.cell *= (v / atoms.get_volume())**(1 / 3)
atoms.get_potential_energy()

#Test commit push on second laptop

## Adsorb one C atom using built-in BFGS.

### First prepare the supercell (so the atom adsorbate does not see its mirror image).

In [ ]:
# Now prepare adsorption.
from ase.build import add_adsorbate, fcc111
from ase.visualize import view
ads = 'C'
n_layers = 3
a = atoms.cell[0, 1] * 2 # Equilibrium lattice constant.
atoms = fcc111("Cu", (4, 4, n_layers), a=a)
atoms.get_tags()

In [ ]:
view(atoms, viewer='x3d')

In [ ]:
atoms
atoms.get_positions()

## Add single atom adsorbate.

In [ ]:
ads_height = 1.5 #modified initial conditions
add_adsorbate(atoms, ads, height=ads_height, position='fcc')
#defined vacuum layer for no overlap between supercells NEEDS CONVERGENCE STUDY
atoms.center(vacuum = 10, axis = 2) 
atoms.get_tags()
atoms.get_positions()

In [ ]:
view(atoms, viewer='x3d')


In [ ]:
atoms[-1]

In [ ]:
# Constrain all atoms except the adsorbate:
from ase.constraints import FixAtoms
fixed = list(range(len(atoms) - 1))
atoms.constraints = [FixAtoms(indices=fixed)]

## Optimize adsorbate position usgin built-in BFGS from ASE.

In [ ]:
from ase.optimize import BFGS
atoms.calc = EMT()
opt = BFGS(atoms, logfile=None)
opt.run(fmax=0.0001)

In [ ]:
# Final adsorbate position.
### print(atoms[3].position)
print(atoms[-1].position) # modified to get adsorbate position

# Final energy.
print(atoms.get_potential_energy())

## Comparison with BoTorch

In [ ]:
import numpy as np
import torch

In [ ]:
# Do not allow our atom to go inside the surface. 
# Also restrict x-y to the unit cell size.
bulk_z_max = np.max(atoms[:-1].positions[:, 2]) #modified to account for changes in initial conditions + universal
print(bulk_z_max)
cell_x_min, cell_x_max = float(np.min(atoms.cell[:, 0])), float(np.max(atoms.cell[:, 0]))
cell_y_min, cell_y_max = float(np.min(atoms.cell[:, 1])), float(np.max(atoms.cell[:, 1]))
#z_adsorb_max = 3 * ads_height
z_adsorb_max = atoms[-1].position[-1] + 5 # modified to account for changes in initial conditions

## Set up evaluation function (pipe to ASE) for trial parameters suggested by Ax. 
Note that this function can return additional keys that can be used in the `outcome_constraints` of the experiment.

In [ ]:
def evaluate(parameters):
    x = np.array([parameters.get(f"x"), parameters.get(f"y"), parameters.get(f"z")])
     # Can put zeros since constraints are respected by set_positions.
    new_pos = np.vstack([np.zeros((atoms.get_number_of_atoms() - 1, 3)), x])
    atoms.set_positions(new_pos, apply_constraint=True)
    energy = atoms.get_potential_energy()
    
    # In our case, standard error is 0, since we are computing a synthetic function.
    return {"adsorption_energy": (energy, 0.0)} # We have 0 noise on the target.

In [ ]:
from botorch.models import SingleTaskGP, ModelListGP, FixedNoiseGP
# Ax wrappers for BoTorch components
from ax.models.torch.botorch_modular.model import BoTorchModel
from ax.models.torch.botorch_modular.surrogate import Surrogate
from gpytorch.mlls.exact_marginal_log_likelihood import ExactMarginalLogLikelihood
from botorch.acquisition.analytic import ExpectedImprovement

# model = SingleTaskGP(init_x, init_y)
# mll = ExactMarginalLogLikelihood(single_model.likelihood, single_model)

# Fit on init data. 
# from botorch import fit_gpytorch_model
# fit_gpytorch_model(mll)

model = BoTorchModel(
    # Optional `Surrogate` specification to use instead of default
    surrogate=Surrogate(
        # BoTorch `Model` type
        botorch_model_class=FixedNoiseGP,
        # Optional, MLL class with which to optimize model parameters
        mll_class=ExactMarginalLogLikelihood,
        # Optional, dictionary of keyword arguments to underlying
        # BoTorch `Model` constructor
        model_options={},
    ),
    # Optional BoTorch `AcquisitionFunction` to use instead of default
    botorch_acqf_class=ExpectedImprovement,
    # Optional dict of keyword arguments, passed to the input
    # constructor for the given BoTorch `AcquisitionFunction`
    acquisition_options={},
)

## Create client and initial sampling strategy to warm-up the GP model

In [ ]:
import torch
from ax.service.ax_client import AxClient
from ax.service.utils.instantiation import ObjectiveProperties

from ax.modelbridge.generation_strategy import GenerationStep, GenerationStrategy
from ax.modelbridge.registry import Models


gs = GenerationStrategy(
    steps=[
        # 1. Initialization step (does not require pre-existing data and is well-suited for
        # initial sampling of the search space)
        GenerationStep(
            model=Models.SOBOL,
            num_trials=5,  # How many trials should be produced from this generation step
            min_trials_observed=3,  # How many trials need to be completed to move to next model
            max_parallelism=5,  # Max parallelism for this step
            model_kwargs={"seed": 999},  # Any kwargs you want passed into the model
            model_gen_kwargs={},  # Any kwargs you want passed to `modelbridge.gen`
        ),
        # 2. Bayesian optimization step (requires data obtained from previous phase and learns
        # from all data available at the time of each new candidate generation call)
        GenerationStep(
            model=Models.BOTORCH_MODULAR,
            num_trials=-1,  # No limitation on how many trials should be produced from this step
            max_parallelism=3,  # Parallelism limit for this step, often lower than for Sobol
            # More on parallelism vs. required samples in BayesOpt:
            # https://ax.dev/docs/bayesopt.html#tradeoff-between-parallelism-and-total-number-of-trials
        ),
    ]
)

# Initialize the client - AxClient offers a convenient API to control the experiment
ax_client = AxClient(generation_strategy=gs)

## Set up the optimization experiment in Ax.

In [ ]:
ax_client.create_experiment(
    name="adsorption_experiment",
    parameters=[
        {
            "name": "x",
            "type": "range",
            "bounds": [float(cell_x_min), float(cell_x_max)],
        },
        {
            "name": "y",
            "type": "range",
            "bounds": [float(cell_y_min), float(cell_y_max)],
        },
        {
            "name": "z",
            "type": "range",
            "bounds": [float(bulk_z_max), float(z_adsorb_max)], #I made a modification here, switched both bounds.
        },
    ],
    objectives={"adsorption_energy": ObjectiveProperties(minimize=True)},
    # parameter_constraints=["x1 + x2 <= 2.0"],  # Optional.
    # outcome_constraints=["l2norm <= 1.25"],  # Optional.
)

## Run the BO loop.

In [ ]:
N_BO_steps = 40
for i in range(N_BO_steps):
    parameters, trial_index = ax_client.get_next_trial()
    # Local evaluation here can be replaced with deployment to external system.
    ax_client.complete_trial(trial_index=trial_index, raw_data=evaluate(parameters))

## Display Results

In [ ]:
ax_client.get_trials_data_frame()

## Plot Evolution of adsorption energy.

In [ ]:
from ax.utils.notebook.plotting import render
# from botorch.acquisition

render(ax_client.get_optimization_trace(objective_optimum=0.0))

## Plot learned response surface.

In [ ]:
from ax.plot.contour import interact_contour
model = ax_client.generation_strategy.model
render(interact_contour(model=model, metric_name="adsorption_energy",
                       slice_values={'x': 1.263480218001716, 'y': 1.0, 'z': 3.01}))

In [ ]:
ax_client.get_best_parameters()
#Modify atoms to represent the best solution
params = ax_client.get_best_parameters()[:1][0]
atoms[-1].position[:] = params['x'],params['y'],params['z']

## Visualize the resulting chemical system.

In [ ]:
from ase.visualize import view
view(atoms, viewer='x3d')

In [ ]:
import matplotlib.pyplot as plt
from ase.visualize.plot import plot_atoms

fig, ax = plt.subplots()

plot_atoms(atoms, ax, radii=0.5, rotation=('90x,45y,0z'))
#plot_atoms(atoms, ax, radii=0.5, rotation=('0x,0y,0z'))


fig.savefig("ase_slab.png")